In [10]:
%pip install scikeras


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import json
import numpy as np
import tensorflow as tf
import time
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam




In [12]:
# Cargar el dataset desde el archivo .npy
path2 = '../DatasetVGG19/train.npy'

In [13]:
data_train = np.load(path2)

In [14]:

def unpack(array):
    x = array[:,:-2]
    y = array[:,-2:]
    y = np.argmax(y, axis=1)
    return x,y

def model_score_tf(X,y,model,dataset):
    test_loss, test_accuracy = model.evaluate(X, y)
    print(f"Precisión del modelo {dataset}: {test_accuracy * 100:.2f}%")
    return test_accuracy

def writeJson(tiempo,accu,metod):
    nuevo_registro = {
        'Tipo Ejecucion' : "GPU_cross",
        'Metodo' : metod,
        'Epochs':20,
        'tiempo_ejecucion': tiempo,
        'accuracy' : accu,
        'paralelism' : 4
    }
    nombre_archivo = '../GPU_tiempos.json'

    # Leer el contenido existente, si el archivo ya existe
    if os.path.exists(nombre_archivo):
        with open(nombre_archivo, 'r') as archivo:
            try:
                registros = json.load(archivo)
            except json.JSONDecodeError:
                registros = []
    else:
        registros = []

    # Agregar el nuevo registro
    registros.append(nuevo_registro)

    # Guardar los registros actualizados en el archivo JSON
    with open(nombre_archivo, 'w') as archivo:
        json.dump(registros, archivo, indent=4)

In [15]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# MLPClassifier

In [16]:

X_train,y_train = unpack(data_train)

# Convertir las etiquetas a formato one-hot para TensorFlow
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)

In [17]:
def create_model(learning_rate=0.01, hidden_activation='relu', output_activation='sigmoid', layers=(500, 200, 2)):
    layers_arr = []

    for i in range(len(layers)):
        if i == 0:
            layers_arr.append(Dense(layers[i], activation=hidden_activation, input_shape=(1000,)))
        elif len(layers)-1:
            layers_arr.append(Dense(layers[i], activation=output_activation))
        else:
            layers_arr.append(Dense(layers[i], activation=hidden_activation))

    model = Sequential(layers_arr)
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                loss='binary_crossentropy', 
                metrics=['accuracy'])
    return model

In [18]:
print("Entrenamiento MLP Classifier")

param_grid = {
    'layers': [(500, 2), (100, 2),(500,  200, 2)],
    'learning_rate': [0.001,0.0001]
}
model = KerasClassifier(build_fn=create_model,learning_rate=0.01,  epochs=20, verbose=0, hidden_activation='relu', output_activation='sigmoid', layers=(500, 200, 2))
inicio = time.time()

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=2)
grid_result = grid.fit(X_train,y_train)

fin = time.time()
tiempo_ejecucion_MLPC = fin - inicio

print(f"El tiempo de ejecucion fue de {tiempo_ejecucion_MLPC:.2f} segundos")

Entrenamiento MLP Classifier


2024-10-29 22:45:32.203819: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 22:45:32.203816: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 22:45:32.213644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 22:45:32.213834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been reg

El tiempo de ejecucion fue de 332.26 segundos


In [21]:
writeJson(tiempo_ejecucion_MLPC,grid_result.best_score_,"MLPC")

# Tiempos

In [22]:
print(f"Tiempo MLPC = {tiempo_ejecucion_MLPC:.2f} segundos")

Tiempo MLPC = 332.26 segundos
